# Baseline for Multi-Facility Location Problem

This notebook demonstrates how to:
1. Load the provided `.pkl` files for train and test.
2. Implement a simple baseline approach to place multiple facilities.
3. Evaluate the average social cost.

We assume:
- We have `all_data_train.pkl` and `all_data_test.pkl` in a local `data/` folder.
- A minimal baseline that places facilities at certain quantiles of the agent peaks.


In [ ]:
import os
import pickle
import numpy as np
import pandas as pd

# For simple numeric display
pd.set_option('display.precision', 6)

# Parameters:
DATA_TRAIN_PATH = 'data/all_data_train.pkl'
DATA_TEST_PATH = 'data/all_data_test.pkl'

# Number of facilities:
K = 2  # You can change this to 1, 3, etc.

###########################################################
# 1. LOAD DATA
###########################################################
with open(DATA_TRAIN_PATH, 'rb') as f:
    data_train = pickle.load(f)
with open(DATA_TEST_PATH, 'rb') as f:
    data_test = pickle.load(f)

print("Loaded train distributions:", list(data_train.keys()))
print("Loaded test distributions: ", list(data_test.keys()))

Each entry in `data_train` / `data_test` is keyed by `(distribution, n)` where:
- **distribution** ∈ \{`uniform`, `normal`, `beta1`, `beta2`\}
- **n** is the number of agents.

Each value is a dictionary containing:
- **`peaks`**: shape `(1000, n)` — 1000 samples, each row is the real peaks of n agents.
- **`misreports`**: shape `(10000, n)` — 10 misreports per sample (1000 * 10 = 10000). We won't use misreports here in this baseline, but it is available if you'd like to measure strategy-proofness.


## 2. Baseline Approach: Quantile Placement

We'll define a very simple function that:
1. Given an array of agent peaks (length `n`), sorts them.
2. Chooses `K` quantile positions.
3. Places each facility there.

> For example, if `K=2`, we might place them at the 1/3 and 2/3 quantiles of the agent peaks. That is arbitrary but easy to implement.

Then for each sample (each row in `peaks`), we compute the social cost = sum of each agent's distance to the nearest facility.

In [ ]:
def baseline_facility_location(peaks, K=2):
    """
    Place K facilities at K equally spaced quantiles.
    peaks: array of shape (n,) for a single sample.
    """
    sorted_peaks = np.sort(peaks)
    n = len(peaks)
    facilities = []
    for i in range(K):
        # compute the quantile fraction
        frac = (i + 1) / (K + 1)  
        # find approximate quantile index
        idx = int(frac * (n - 1))
        facilities.append(sorted_peaks[idx])
    return np.array(facilities)

def compute_social_cost(peaks, facilities):
    """
    Given an array of agent peaks (shape (n,)), and an array of facility positions, 
    compute the sum of distances from each agent to its nearest facility.
    """
    # for each agent, distance to each facility, take min
    distances = [ np.abs(p - facilities).min() for p in peaks ]
    return np.sum(distances)

def evaluate_baseline(peaks_array, K=2):
    """
    Evaluate the baseline approach on an entire set of samples.
    peaks_array: shape (num_samples, n)
    """
    total_cost = 0.0
    num_samples = peaks_array.shape[0]
    for i in range(num_samples):
        peaks = peaks_array[i]
        facs = baseline_facility_location(peaks, K)
        cost = compute_social_cost(peaks, facs)
        total_cost += cost
    return total_cost / num_samples  # average cost per sample


## 3. Evaluate on Train & Test

We'll iterate over a small subset of distributions and agent counts to show how to compute the baseline social cost.

In [ ]:
# Let's pick some (distribution, n) keys to test.
keys_to_check = [
    ('uniform', 5),
    ('normal', 5),
    ('beta1', 5),
    ('beta2', 5),
    # You can add more if you want.
]

results = []
for dist_n in keys_to_check:
    if dist_n not in data_train:
        continue
    peaks_train = data_train[dist_n]['peaks']  # shape (1000, n)
    peaks_test = data_test[dist_n]['peaks']    # shape (1000, n)

    # Evaluate on train
    avg_cost_train = evaluate_baseline(peaks_train, K=K)

    # Evaluate on test
    avg_cost_test = evaluate_baseline(peaks_test, K=K)

    results.append({
        'distribution': dist_n[0],
        'n': dist_n[1],
        'K': K,
        'avg_cost_train': avg_cost_train,
        'avg_cost_test': avg_cost_test
    })

df_results = pd.DataFrame(results)
df_results

The table above shows the **average cost** (distance) over 1000 samples in train and test sets, using this naive quantile-based placement.

## 4. Notes
- This code does **not** consider misreports, so it doesn't measure strategy-proofness.
- The baseline we used is extremely simple. You can improve it by:
  - Trying other quantile placements.
  - Using a clustering method (like k-means on 1D) to find facility positions.
  - Minimizing weighted cost if there are agent weights.
  - Analyzing misreports to see if the facility location can be manipulated.
